# NLP on Yelp Open Dataset for Review Classification

Questo notebook ha il compito di effettuare tramite NLP una classificazione delle reviews in positive e negative andando ad analizzare testi tokenizzati all'interno del dataset fornito da Open Yelp Dataset. Ci soffermeremo solamente sulla tabella dedicata alla review poichè si giudicano sufficiente, per raggiungere un livello di precisione accettabile, le informazioni contenute all'interno delle colonne della tabella review fornita in input.

### Import Libraries

In [ ]:
# librerie di default
import pandas as pd
import numpy as np

# librerie per il data analysis
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

# librerie per il text manipulation
import nltk as nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS, strip_non_alphanum
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

from collections import Counter, defaultdict
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer
# librerie per il data modelling
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 1. Data Loading

La fase di data loading non fa altro che caricare all'interno di un pandas dataframe le informazioni contenute nel dataset JSON in formato tabellare per poi utilizzarlo nella fase di data analysis per visualizzare le correlazioni e i valori al suo interno tramite visualizzazione grafica in modo da rendersi conto di che tipo e in che quantità sono distribuiti i dati.

In [ ]:
# definiamo i tipi degli attributi JSON per l'attributo dtype di read_json
rtypes = {  "review_id": str,
            "user_id":str,
            "business_id":str,
            "stars": np.float16, 
            "useful": np.int32, 
            "funny": np.int32,
            "cool": np.int32,
            "text" : str,
           }

# file path del dataset json
path = './data/yelp_academic_dataset_review.json'

# grandezza dei chunk
chunkSize = 100000

In [ ]:
%%time
# creazione del JsonReader
review = pd.read_json(path, lines=True,
                      dtype=rtypes,
                      chunksize=chunkSize)
chunkList = []

# utilizzo della segmentazione in chunk per creare dal JsonReader il dataframe
for chunkReview in review:
    # rimozione degli attributi id
    chunkReview = chunkReview.drop(['review_id', 'user_id','business_id'], axis=1)
    chunkList.append(chunkReview)
    
# concatenazione degli elementi nella chunkList per righe
df = pd.concat(chunkList, ignore_index=True, axis=0)

In [ ]:
# visualizzazione degli elementi in testa
df.head()

## 2. Data Analysis

Durante la fase di data analysis andremo ad ispezionare il dataframe caricato andando a visualizzare graficamente come sono distribuiti i valori associati ad ogni attributo.

In [ ]:
# informazioni sulle colonne del dataframe e su quante entries o righe si hanno
df.info()

### 2.1 Stars Analysis

In [ ]:
# definire la grandezza della figura
plt.figure(figsize=(8,8))

# contare i vari valori di stars e visualizzarli su un diagramma a torta
df['stars'].value_counts().plot.pie(startangle=60)

# definire il titolo del plot
plt.title('Distribuzione dei valori per l\'attributo stars')

Le quantità di recensioni, classificate in base al numero di stelle assegnate, è sbilanciata. Si ha un maggior numero per le recensioni con 5 e 4 stelle rispetto a quelle con 1, 2 o 3 stelle.

In [ ]:
# distribuzione dei valori in reviews positive e negative
binstars = pd.DataFrame()
binstars['stars'] = [0 if star <= 3.0 else 1 for star in df['stars']]
# definire la grandezza della figura
plt.figure(figsize=(8,8))


# contare i vari valori di stars e visualizzarli su un diagramma a torta
binstars['stars'].value_counts().plot.pie(startangle=60)

# definire il titolo del plot
plt.title('Distribuzione dei valori positivi e negativi')

### 2.2 Cool, Fun and Useful Analysis

In [ ]:
# definire le correlazioni
corr = df.corr()

# generazione dell'heatmap
sns.heatmap(corr)

Non sono presenti particolari correlazioni forti tra i funny, useful e cool con i valori dati a stars.

### 2.3 Text Analysis

In [ ]:
%%time

# definisce un sottoinsieme delle righe del dataset
subset = df[:100000]
# concatenazione dei testi di ogni riga in una singola stringa
inputText = ' '.join(subset['text']).lower()

# creazione di un wordcloud andando ad ignorare le stopwords
wordCloud = WordCloud(background_color='white', stopwords=STOPWORDS).generate(inputText)
# setting della visualizzazione utilizzando una interpolazione bilineare
plt.imshow(wordCloud, interpolation='bilinear')

# rimozione degli assi
plt.axis('off')
# visualizzazione del wordcloud rappresentante le parole più usate nel testo di una recensione
plt.show()

In [ ]:
# calcolo della frequenza dei termini più utilizzati
wordTokens = word_tokenize(inputText)
tokens = list()
for word in wordTokens:
    if word.isalpha() and word not in STOPWORDS:
        tokens.append(word)
tokenDist = FreqDist(tokens)
# per questioni di visualizzazione, andiamo a prendere solamente i primi 20 termini utilizzati
dist = pd.DataFrame(tokenDist.most_common(20),columns=['term', 'freq'])

In [ ]:
# rappresentazione grafica dei risultati
fig = plt.figure(figsize=(14,8))
ax = fig.add_axes([0,0,1,1])
x = dist['term']
y = dist['freq']
ax.bar(x,y)
plt.title('Frequenza dei termini più utilizzati')
plt.show()

In [ ]:
# Aggiunta di una feature per l'analisi della lunghezza dei testi
df['textLength']  = df['text'].str.len()

In [ ]:
df.head()

In [ ]:
# Differenziazione della lunghezza dei testi in relazione alla valutazione data a stars
graph = sns.FacetGrid(data=df,col='stars')
graph.map(plt.hist,'textLength',bins=50,color='blue')

## 3. Data Pre-processing

Durante la fase di pre-processing, andiamo a pulire e bilanciare il dataframe in modo da poterlo utilizzare per il data modelling.

### 3.1 Cancellazione Colonne

In [ ]:
# cancellazione delle caratteristiche cool, funny, useful e textLength poichè non hanno correlazioni con stars.
df = df.drop(['cool', 'funny', 'useful', 'textLength'], axis=1)

In [ ]:
df.head()

In [ ]:
# rimozione di possibili testi vuoti
df['text'].dropna(inplace=True)

In [ ]:
# ridurre la forma delle parole in minuscolo
df['text'] = [review_text.lower() for review_text in df['text']]

In [ ]:
df['text'].head()

### 3.2 Stars Polarization

In [ ]:
# polarizzazione delle valutazioni a stars in due categorie: 1 = positiva, 0 = negativa

# isoliamo la colonna di testo del dataframe in texts
texts =  df['text']

# andiamo ad impostare negative tutte le recensioni con 3 o meno stelle e positive quelle con 4 e 5 stelle.
stars = [0 if star <= 3.0 else 1 for star in df['stars']]

balancedTexts = [] # rappresenta la collezione di testi presi in considerazione dal dataframe di input
balancedLabels = [] # rappresenta il nuovo valore polarizzato assegnato all'entry (0,1)

# andiamo a bilanciare il dataset andando a dividere recensioni positive e negative con limite di 1.000.000 per categoria
limit = 100000  

# posizione 0 per conteggio di recensioni negative, posizione 1 per quelle positive
negPosCounts = [0, 0] 

for i in range(0,len(texts)):
    polarity = stars[i]
    if negPosCounts[polarity] < limit: # se non si è raggiunto il limite per la categoria di polarizzazione
        balancedTexts.append(texts[i])
        balancedLabels.append(stars[i])
        negPosCounts[polarity] += 1

In [ ]:
df_balanced = pd.DataFrame()
df_balanced['text'] = balancedTexts
df_balanced['labels'] = balancedLabels
df_balanced.head()

In [ ]:
# verifica del conteggio
counter = Counter(df_balanced['labels'])
print(f'Ci sono {counter[1]} recensioni positive e {counter[0]} recensioni negative')

### 3.3 Text Tokenization

In [ ]:
# tokenization
df_balanced['text'] = [word_tokenize(text) for text in df_balanced['text']]

In [ ]:
# stop words
stop_words = np.array(STOPWORDS)
print(stop_words)

### 3.4 Text Cleaning

In [ ]:
%%time
# rimozione delle stop words
df_balanced['text'] = [word for word in df_balanced['text'] if not word in stop_words]
temp_texts = [] 
temp_words = []
# rimozione delle parole non alfanumeriche
for text in df_balanced['text']:
    for word in text:
        if word.isalnum():
            temp_words.append(word)
    temp_texts.append(temp_words)
    temp_words = []

In [ ]:
df_balanced['text'] = temp_texts
df_balanced['text'].head()

In [ ]:
# Checking sulle compile flags di tensorflow
print(tf.sysconfig.get_compile_flags())
print(tf.__version__)

In [ ]:
%%time
# definizione di un tokenizer di 5.000 parole prese dal dataframe
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_balanced['text'])
# trasformazione del testo in sequenze di interi in modo da valutare più velocemente le parole
sequences = tokenizer.texts_to_sequences(df_balanced['text'])
# Sequenze di massimo 200 unità. Se vi sono testi con sequenze più lunghe esse vengono troncate, altrimenti si avrà 
# un riempimenti di 0 per testi undersized.
text_sequence = pad_sequences(sequences, maxlen=200)
labels = np.array(df_balanced['labels'])

In [ ]:
# train and test splitting
x_train, x_test, y_train, y_test = train_test_split(text_sequence , labels ,test_size=0.5, shuffle=True)

In [ ]:
# creazione di un modello sequenziale vuoto in cui aggiungere i vari layers
model_lstm = keras.Sequential()

# aggiunta dei layers
model_lstm.add(layers.Embedding(25000, 128, input_length=250))
model_lstm.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)) # dropout dentro il layer LSTM
model_lstm.add(layers.Dense(1, activation='sigmoid'))

model_lstm.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

model_lstm.summary()

In [ ]:
results_lstm = model_lstm.fit(x_train, y_train, validation_split=0.33, epochs=3)

In [ ]:
model_lstm.evaluate(x_test, y_test)

modelLSTM.evaluate(xTest, yTest)

In [ ]:
model_lstm_v2 = keras.Sequential()
model_lstm_v2.add(layers.Embedding(25000, 128, input_length=250))
model_lstm_v2.add(layers.Dropout(0.5)) # layer di dropout esterno in seguito ad Embedding
model_lstm_v2.add(layers.Conv1D(64, 5, activation='relu'))
model_lstm_v2.add(layers.MaxPooling1D(pool_size=4))
model_lstm_v2.add(layers.LSTM(128))
model_lstm_v2.add(layers.Dense(1, activation='sigmoid'))


model_lstm_v2.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

model_lstm_v2.summary()

In [ ]:
model_lstm_v2.fit(x_train, y_train, validation_split=0.33, epochs=3)

In [ ]:
model_lstm_v2.evaluate(x_test, y_test)

In [ ]:
model_bid = keras.Sequential()
model_bid.add(layers.Embedding(25000, 128, input_length=250))
model_bid.add(layers.Dropout(0.5))
model_bid.add(layers.Conv1D(64, 5, activation='relu'))
model_bid.add(layers.Bidirectional(layers.LSTM(128)))
model_bid.add(layers.Dense(1, activation='sigmoid'))
model_bid.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

model_bid.summary()

In [ ]:
model_bid.fit(x_train, y_train, validation_split=0.33, epochs=3)

In [ ]:
model_bid.evaluate(x_test,y_test)

## 5. Model Testing

In [ ]:
## 4.2 Save model

In [ ]:
import pickle

# salviamo il tokenizer e i modelli su file
with open("dump/keras_tokenizer.pickle", "wb") as f:
   pickle.dump(tokenizer, f)
with open("dump/tokenizer/yelp_model_lstm.hdf5","wb") as lstm_file:
    model_lstm.save(lstm_file)
          
with open("dump/model/yelp_model_lstm_v2.hdf5", "wb") as lstm_v2_file:
    model_lstm_v2.save(lstm_v2_file)
    
with open("dump/model/yelp_bidirectional.hdf5", "wb") as bid_file:
    model_bid.save(bid_file)

In [ ]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle

# carichiamo il tokenizer e il modello da file
with open("keras_tokenizer.pickle", "rb") as f:
   tokenizer = pickle.load(f)

# TODO: load other models
model_lstm = load_model("dump/model/yelp_model_lstm.hdf5")
model_lstm_v2 = load_model("dump/model/yelp_model_lstm_v2.hdf5")
model_bid = load_model("dump/model/yelp_bidirectional.hdf5")

# definiamo gli esempi su cui testare il modello
examples_reviews = ["slow orders but good food", "Delicious foods! Awesome!", "Bad food, bad people... horrible!"]

# usiamo il tokenizer per creare sequenze di interi da dare al modello
sequences = tokenizer.texts_to_sequences(examples_reviews)
data_examples = pad_sequences(sequences, maxlen=250)

# effettuare le predizioni e stampare i risultati
predictions_lstm = model_lstm.predict(data_examples)
predictions_lstm_v2 = model_lstm_v2.predict(data_examples)
predictions_bid = model_bid.predict(data_examples)

print(f"Risultati model_lstm: {predictions_lstm}\n"+
    f"Risultati model_lstm_v2: {predictions_lstm_v2}\n" + 
      f"Risultati model_bid: {predictions_bid}")

In [ ]:
# TODO: Data Analytics sui risultati